In [1]:
import pathlib

import pandas as pd
import toml

In [2]:
output_path = pathlib.Path("../IDR_metadata/screenA/")
output_path.mkdir(exist_ok=True, parents=True)
output_path = output_path / "idr0000-screenA-processed.txt"

In [3]:
ground_truth_path = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
)

# read the ground truth
ground_truth = toml.load(ground_truth_path)
apoptosis = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control = ground_truth["Healthy"]["healthy_groups_list"]

In [4]:
# path to the data (processed)
PBMC_path = pathlib.Path(
    "../../data/PBMC_preprocess_sc_norm_no_fs_aggregated_nomic.parquet"
)
SHSY5Y_path = pathlib.Path(
    "../../data/SHSY5Y_preprocess_sc_norm_no_fs_aggregated_nomic.parquet"
)
# read the data
PBMC = pd.read_parquet(PBMC_path)
SHSY5Y = pd.read_parquet(SHSY5Y_path)

In [5]:
print(SHSY5Y.shape, PBMC.shape)
SHSY5Y["Metadata_Well"].unique()

(154, 2849) (154, 2849)


array(['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21',
       'B22', 'B23', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
       'C20', 'C21', 'C22', 'C23', 'D13', 'D14', 'D15', 'D16', 'D17',
       'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'E13', 'E14', 'E15',
       'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'F13',
       'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22',
       'F23', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20',
       'G21', 'G22', 'G23', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18',
       'H19', 'H20', 'H21', 'H22', 'H23', 'I13', 'I14', 'I15', 'I16',
       'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23', 'J13', 'J14',
       'J15', 'J16', 'J17', 'J18', 'J19', 'J20', 'J21', 'J22', 'J23',
       'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20', 'K21',
       'K22', 'K23', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', 'L19',
       'L20', 'L21', 'L22', 'L23', 'M13', 'M14', 'M15', 'M16', 'M17',
       'M18', 'M19',

In [6]:
PBMC["ground_truth"] = "other"
PBMC["apoptosis"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(apoptosis)
PBMC["pyroptosis"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    pyroptosis
)
PBMC["control"] = PBMC["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(control)

# merge the apoptosis, pyroptosis and control columns into the ground_truth column
PBMC.loc[PBMC["apoptosis"], "ground_truth"] = "apoptosis"
PBMC.loc[PBMC["pyroptosis"], "ground_truth"] = "pyroptosis"
PBMC.loc[PBMC["control"], "ground_truth"] = "control"

# drop the apoptosis, pyroptosis and control columns
PBMC = PBMC.drop(
    columns=[
        "apoptosis",
        "pyroptosis",
        "control",
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
    ]
)


SHSY5Y = pd.read_parquet(SHSY5Y_path)
# read the ground truth
ground_truth = toml.load(ground_truth_path)

apoptosis = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control = ground_truth["Healthy"]["healthy_groups_list"]
# make a column for the ground truth
SHSY5Y["ground_truth"] = "other"
SHSY5Y["apoptosis"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    apoptosis
)
SHSY5Y["pyroptosis"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(
    pyroptosis
)
SHSY5Y["control"] = SHSY5Y["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].isin(control)

# merge the apoptosis, pyroptosis and control columns into the ground_truth column
SHSY5Y.loc[SHSY5Y["apoptosis"], "ground_truth"] = "apoptosis"
SHSY5Y.loc[SHSY5Y["pyroptosis"], "ground_truth"] = "pyroptosis"
SHSY5Y.loc[SHSY5Y["control"], "ground_truth"] = "control"

# drop the apoptosis, pyroptosis and control columns
SHSY5Y = SHSY5Y.drop(
    columns=[
        "apoptosis",
        "pyroptosis",
        "control",
        "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
    ]
)

# concat the two dataframes
data = pd.concat([PBMC, SHSY5Y], axis=0)
data.head()
data.reset_index(drop=True, inplace=True)
data

,Metadata_Well,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Center_Y,Cytoplasm_AreaShape_Compactness,...,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256,ground_truth
0,B02,0.119709,0.129981,0.023535,0.012792,0.021824,0.011274,0.022585,0.012096,-0.032252,...,-0.012993,-0.010951,-0.011109,-0.011178,-0.010534,0.015620,0.016167,0.015749,0.015758,pyroptosis
1,B03,0.095616,0.096278,0.010884,0.029580,0.009907,0.028735,0.010267,0.029245,-0.033178,...,-0.026522,-0.011869,-0.012319,-0.012108,-0.011691,0.009975,0.010342,0.010703,0.010207,pyroptosis
2,B04,-0.031231,-0.046841,-0.042766,0.055657,-0.041048,0.057418,-0.042007,0.056635,-0.095278,...,0.006014,0.034417,0.035198,0.035325,0.034269,-0.037674,-0.037733,-0.037377,-0.037603,pyroptosis
3,B05,-0.024048,-0.037242,-0.058885,0.039394,-0.057277,0.040976,-0.058201,0.040268,-0.081000,...,-0.019757,0.051339,0.051311,0.051653,0.050933,-0.032363,-0.032613,-0.032380,-0.032385,pyroptosis
4,B06,-0.038195,-0.035832,0.009600,0.017445,0.010187,0.018096,0.009889,0.017761,0.022371,...,-0.026637,-0.018511,-0.018996,-0.018858,-0.018147,-0.009723,-0.009940,-0.010195,-0.009590,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,O19,0.059221,0.093592,-0.015885,0.026344,-0.021589,0.020314,-0.018784,0.023570,-0.030737,...,0.019319,-0.033065,-0.034027,-0.033728,-0.033238,0.008525,0.007859,0.008095,0.008473,other
304,O20,0.090042,0.184824,0.009052,-0.012419,-0.001640,-0.023019,0.003463,-0.018012,-0.008031,...,0.039120,0.049804,0.051771,0.051748,0.049845,-0.005626,-0.006105,-0.005761,-0.004413,control
305,O21,0.040557,0.106181,0.012091,0.078303,0.004443,0.070628,0.008757,0.074141,-0.039158,...,0.026257,0.122080,0.127950,0.130005,0.126220,-0.002300,-0.001873,-0.002267,-0.002593,control
306,O22,0.040488,0.031649,0.025968,0.033714,0.024841,0.029542,0.025255,0.031351,-0.078584,...,-0.005449,0.015558,0.017991,0.017287,0.014584,-0.020307,-0.020023,-0.020237,-0.020045,control


In [7]:
# write the data to a txt file
data.to_csv(output_path, sep="\t", index=False)